CRUD - CREATE, READ, UPDATE DELETE

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 31.2 MB/s eta 0:00:00


In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
host = '34.70.26.188' # endereço do banco externo que eu quero acessar
user = 'root'
password = ''
database = 'locadora'

In [ ]:
!curl ipecho.net/plain

34.86.203.67

In [ ]:
def consulta(query, tabela):
  connection = mysql.connector.connect(
      host = host,
      user = user,
      password = password,
      database = database
  )
  cursor = connection.cursor()

  try:
    cursor.execute(query)
    result = cursor.fetchall()
    globals()[tabela] = pd.DataFrame(result, columns=cursor.column_names)
    display(globals()[tabela])
  finally:
    cursor.close()
    connection.close()

In [ ]:
def ddl(comando):
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = connection.cursor()

    try:
        cursor.execute(comando)
        connection.commit()  # Confirma as mudanças no banco
        print(f"Comando DDL executado com sucesso:\n{comando}")
    except mysql.connector.Error as err:
        print(f"Erro ao executar o comando DDL: {err}")
    finally:
        cursor.close()
        connection.close()

In [ ]:
consulta('''
SHOW TABLES;
''',''' ''')

,Tables_in_locadora
0,aluguel
1,carro
2,cliente
3,clientes_modelos
4,detalhes_aluguel
5,marca
6,marcas_frequencia


# C do CRUD

## Criação e Modificação de Tabelas

In [ ]:
# Criar tabela pagamentos
ddl('''
CREATE TABLE pagamentos (
    codpagamento INT PRIMARY KEY,
    codaluguel INT,
    valor FLOAT,
    data_pagamento DATE,
    FOREIGN KEY (codaluguel) REFERENCES aluguel (codaluguel)
);
''')

Comando DDL executado com sucesso:

CREATE TABLE pagamentos (
    codpagamento INT PRIMARY KEY,
    codaluguel INT,
    valor FLOAT,
    data_pagamento DATE,
    FOREIGN KEY (codaluguel) REFERENCES aluguel (codaluguel)
);



In [ ]:
consulta('''
SHOW TABLES;
''',''' ''')

,Tables_in_locadora
0,aluguel
1,carro
2,cliente
3,marca
4,pagamentos


In [ ]:
# Adicionar coluna email em cliente
ddl('''
ALTER TABLE cliente ADD COLUMN email VARCHAR(255);
''')

Comando DDL executado com sucesso:

ALTER TABLE cliente ADD COLUMN email VARCHAR(255);



In [ ]:
consulta('''
DESC cliente;
''','''''')

,Field,Type,Null,Key,Default,Extra
0,codcliente,int,NO,PRI,None,
1,nome,varchar(50),YES,,None,
2,cidade,varchar(50),YES,,None,
3,sexo,char(1),YES,,None,
4,estado,char(2),YES,,None,
5,estadocivil,char(1),YES,,None,
6,email,varchar(255),YES,,None,


In [ ]:
# Alterar tipo da coluna valor em carro
ddl('''
ALTER TABLE carro MODIFY COLUMN valor DECIMAL(10,2);
''')

Comando DDL executado com sucesso:

ALTER TABLE carro MODIFY COLUMN valor DECIMAL(10,2);



In [ ]:
consulta('''
DESC carro;
''','''''')

,Field,Type,Null,Key,Default,Extra
0,codcarro,int,NO,PRI,None,
1,codmarca,int,YES,MUL,None,
2,modelo,varchar(30),NO,,None,
3,valor,"decimal(10,2)",YES,,None,


In [ ]:
# Renomear coluna estado para uf em cliente
ddl('''
ALTER TABLE cliente CHANGE COLUMN estado uf CHAR(2);
''')

Comando DDL executado com sucesso:

ALTER TABLE cliente CHANGE COLUMN estado uf CHAR(2);



In [ ]:
consulta('''
DESC cliente;
''','''''')

,Field,Type,Null,Key,Default,Extra
0,codcliente,int,NO,PRI,None,
1,nome,varchar(50),YES,,None,
2,cidade,varchar(50),YES,,None,
3,sexo,char(1),YES,,None,
4,uf,char(2),YES,,None,
5,estadocivil,char(1),YES,,None,
6,email,varchar(255),YES,,None,


In [ ]:
# Adicionar NOT NULL na coluna marca em marca
ddl('''
ALTER TABLE marca MODIFY COLUMN marca VARCHAR(255) NOT NULL;
''')

Comando DDL executado com sucesso:

ALTER TABLE marca MODIFY COLUMN marca VARCHAR(255) NOT NULL;



In [ ]:
consulta('''
DESCRIBE marca;
''',''' ''')

,Field,Type,Null,Key,Default,Extra
0,codmarca,int,NO,PRI,None,
1,marca,varchar(255),NO,,None,


In [ ]:
# Remover coluna estadocivil em cliente
ddl('''
ALTER TABLE cliente DROP COLUMN estadocivil;
''')

Comando DDL executado com sucesso:

ALTER TABLE cliente DROP COLUMN estadocivil;



In [ ]:
consulta('''
DESCRIBE cliente;
''',''' ''')

,Field,Type,Null,Key,Default,Extra
0,codcliente,int,NO,PRI,None,
1,nome,varchar(50),YES,,None,
2,cidade,varchar(50),YES,,None,
3,sexo,char(1),YES,,None,
4,uf,char(2),YES,,None,
5,email,varchar(255),YES,,None,


In [ ]:
# Excluir tabela pagamentos
ddl('''
DROP TABLE pagamentos;
''')

Comando DDL executado com sucesso:

DROP TABLE pagamentos;



In [ ]:
# Remover coluna data_aluguel em aluguel
ddl('''
ALTER TABLE aluguel DROP COLUMN data_aluguel;
''')

Comando DDL executado com sucesso:

ALTER TABLE aluguel DROP COLUMN data_aluguel;



In [ ]:
# DELETAR Banco locadora
ddl('''
DROP DATABASE locadora;
''')

Comando DDL executado com sucesso:

DROP DATABASE locadora;



# Reconstruíndo o banco locadora

In [ ]:
ddl('''
CREATE TABLE cliente(
codcliente INT PRIMARY KEY,
nome VARCHAR(50),
cidade VARCHAR(50),
sexo CHAR(1),
estado CHAR(2),
estadocivil CHAR(1)
);
''')

Comando DDL executado com sucesso:

CREATE TABLE cliente(
codcliente INT PRIMARY KEY,
nome VARCHAR(50),
cidade VARCHAR(50),
sexo CHAR(1),
estado CHAR(2),
estadocivil CHAR(1)
);



In [ ]:
ddl('''
CREATE TABLE marca(
codmarca INT PRIMARY KEY,
marca VARCHAR(30)
);
''')

Comando DDL executado com sucesso:

CREATE TABLE marca(
codmarca INT PRIMARY KEY,
marca VARCHAR(30)
);



In [ ]:
ddl('''
CREATE TABLE carro(
codcarro INT PRIMARY KEY,
codmarca INT,
modelo VARCHAR(30) NOT NULL,
valor FLOAT,
FOREIGN KEY (codmarca) REFERENCES marca (codmarca)
);
''')

Comando DDL executado com sucesso:

CREATE TABLE carro(
codcarro INT PRIMARY KEY,
codmarca INT,
modelo VARCHAR(30) NOT NULL,
valor FLOAT,
FOREIGN KEY (codmarca) REFERENCES marca (codmarca)
);



In [ ]:
ddl('''
CREATE TABLE aluguel(
codaluguel INT PRIMARY KEY,
codcliente INT,
codcarro INT,
data_aluguel DATE,
FOREIGN KEY (codcliente) REFERENCES cliente (codcliente),
FOREIGN KEY (codcarro) REFERENCES carro (codcarro)
);
''')

Comando DDL executado com sucesso:

CREATE TABLE aluguel(
codaluguel INT PRIMARY KEY,
codcliente INT,
codcarro INT,
data_aluguel DATE,
FOREIGN KEY (codcliente) REFERENCES cliente (codcliente),
FOREIGN KEY (codcarro) REFERENCES carro (codcarro)
);



In [ ]:
ddl('''
INSERT INTO cliente (codcliente, nome, cidade, sexo, estado, estadocivil)
VALUES
(1, 'Ana Silva', 'Duque de Caxias', 'F', 'RJ', 'C'),
(2, 'Bruna Pereira', 'Niterói', 'F', 'RJ', 'C'),
(3, 'Túlio Nascimento', 'Duque de Caxias', 'M', 'RJ', 'S'),
(4, 'Fernando Souza', 'Campinas', 'M', 'SP', 'S'),
(5, 'Lúcia Andrade', 'São Paulo', 'F', 'SP', 'C');
''')

Comando DDL executado com sucesso:

INSERT INTO cliente (codcliente, nome, cidade, sexo, estado, estadocivil)
VALUES
(1, 'Ana Silva', 'Duque de Caxias', 'F', 'RJ', 'C'),
(2, 'Bruna Pereira', 'Niterói', 'F', 'RJ', 'C'),
(3, 'Túlio Nascimento', 'Duque de Caxias', 'M', 'RJ', 'S'),
(4, 'Fernando Souza', 'Campinas', 'M', 'SP', 'S'),
(5, 'Lúcia Andrade', 'São Paulo', 'F', 'SP', 'C');



In [ ]:
ddl('''
INSERT INTO marca (codmarca, marca) VALUES
(1, 'Ford'),
(2, 'Fiat'),
(3, 'Chevrolet'),
(4, 'Volkswagen'),
(5, 'Renault');
''')

Comando DDL executado com sucesso:

INSERT INTO marca (codmarca, marca) VALUES
(1, 'Ford'),
(2, 'Fiat'),
(3, 'Chevrolet'),
(4, 'Volkswagen'),
(5, 'Renault');



In [ ]:
ddl('''
INSERT INTO carro (codcarro, modelo, codmarca, valor) VALUES
(1, 'Ka', 1, 100.00),
(2, 'Argo', 2, 150.00),
(3, 'Onix', 3, 170.00),
(4, 'Polo', 4, 150.00),
(5, 'Kwid', 5, 120.00);
''')

Comando DDL executado com sucesso:

INSERT INTO carro (codcarro, modelo, codmarca, valor) VALUES
(1, 'Ka', 1, 100.00),
(2, 'Argo', 2, 150.00),
(3, 'Onix', 3, 170.00),
(4, 'Polo', 4, 150.00),
(5, 'Kwid', 5, 120.00);



In [ ]:
ddl('''
INSERT INTO aluguel (codaluguel, codcliente, codcarro, data_aluguel)VALUES
(1, 3, 2, '2023-04-01'),
(2, 2, 1, '2023-04-02'),
(3, 2, 1, '2023-04-03'),
(4, 2, 3, '2023-04-04'),
(5, 1, 4, '2023-04-05'),
(6, 1, 4, '2023-04-13'),
(7, 1, 1, '2023-04-15'),
(8, 5, 2, '2023-04-19'),
(9, 5, 2, '2023-04-21'),
(10,3, 1, '2023-04-25');
''')

Comando DDL executado com sucesso:

INSERT INTO aluguel (codaluguel, codcliente, codcarro, data_aluguel)VALUES
(1, 3, 2, '2023-04-01'),
(2, 2, 1, '2023-04-02'),
(3, 2, 1, '2023-04-03'),
(4, 2, 3, '2023-04-04'),
(5, 1, 4, '2023-04-05'),
(6, 1, 4, '2023-04-13'),
(7, 1, 1, '2023-04-15'),
(8, 5, 2, '2023-04-19'),
(9, 5, 2, '2023-04-21'),
(10,3, 1, '2023-04-25');



In [ ]:
# Inserir cliente "João Silva"
ddl('''
INSERT INTO cliente (codcliente, nome, cidade, estado, estadocivil)
VALUES (6, 'João Silva', 'São Paulo', 'SP', 'S');
''')

Comando DDL executado com sucesso:

INSERT INTO cliente (codcliente, nome, cidade, estado, estadocivil)
VALUES (6, 'João Silva', 'São Paulo', 'SP', 'S');



In [ ]:
# Inserir marca "Toyota"
ddl('''
INSERT INTO marca (codmarca, marca)
VALUES (6, 'Toyota');
''')

Comando DDL executado com sucesso:

INSERT INTO marca (codmarca, marca)
VALUES (6, 'Toyota');



In [ ]:
# Inserir carro "Corolla"
ddl('''
INSERT INTO carro (codcarro, codmarca, modelo, valor)
VALUES (6, 6, 'Corolla', 900);
''')

Comando DDL executado com sucesso:

INSERT INTO carro (codcarro, codmarca, modelo, valor)
VALUES (6, 6, 'Corolla', 900);



In [ ]:
#  Inserir aluguel para cliente 1 e carro 2 na data atual
ddl('''
INSERT INTO aluguel (codaluguel, codcliente, codcarro, data_aluguel)
VALUES (11, 1, 2, CURDATE());
''')

Comando DDL executado com sucesso:

INSERT INTO aluguel (codaluguel, codcliente, codcarro, data_aluguel)
VALUES (11, 1, 2, CURDATE());



In [ ]:
# Inserir marca "Mercedes"
ddl('''
INSERT INTO marca (codmarca, marca)
VALUES (7, 'Mercedes');
''')

Comando DDL executado com sucesso:

INSERT INTO marca (codmarca, marca)
VALUES (7, 'Mercedes');



In [ ]:
ddl('''
INSERT INTO carro (codcarro, codmarca, modelo, valor)
VALUES
(7, 7, 'C-280', 60000),
(8, 7, 'E-320', 90000),
(9, 7, 'SLK-320', 70000);
''')

Comando DDL executado com sucesso:

INSERT INTO carro (codcarro, codmarca, modelo, valor)
VALUES 
(7, 7, 'C-280', 60000),
(8, 7, 'E-320', 90000),
(9, 7, 'SLK-320', 70000);



In [ ]:
consulta('''
select * from carro;
''','''''')

,codcarro,codmarca,modelo,valor
0,1,1,Ka,100.0
1,2,2,Argo,150.0
2,3,3,Onix,170.0
3,4,4,Polo,150.0
4,5,5,Kwid,120.0
5,6,6,Corolla,900.0
6,7,7,C-280,60000.0
7,8,7,E-320,90000.0
8,9,7,SLK-320,70000.0


In [ ]:
# Inserir aluguel para cliente 2 e carro 3 na data de ontem
ddl('''
INSERT INTO aluguel (codaluguel, codcliente, codcarro, data_aluguel)
VALUES (12, 2, 3, DATE_SUB(CURDATE(), INTERVAL 1 DAY));
''')

Comando DDL executado com sucesso:

INSERT INTO aluguel (codaluguel, codcliente, codcarro, data_aluguel)
VALUES (12, 2, 3, DATE_SUB(CURDATE(), INTERVAL 1 DAY));



In [ ]:
consulta('''
select * from aluguel;
''','''''')

,codaluguel,codcliente,codcarro,data_aluguel
0,1,3,2,2023-04-01
1,2,2,1,2023-04-02
2,3,2,1,2023-04-03
3,4,2,3,2023-04-04
4,5,1,4,2023-04-05
5,6,1,4,2023-04-13
6,7,1,1,2023-04-15
7,8,5,2,2023-04-19
8,9,5,2,2023-04-21
9,10,3,1,2023-04-25


# UPDATE

In [ ]:
# Atualizar nome do cliente 1
ddl('''
UPDATE cliente
SET nome = 'Ana Silva Ribeiro'
WHERE codcliente = 1;
''')

Comando DDL executado com sucesso:

UPDATE cliente 
SET nome = 'Ana Silva Ribeiro' 
WHERE codcliente = 1;



In [ ]:
consulta('''
select * from cliente;
''','''''')

,codcliente,nome,cidade,sexo,estado,estadocivil
0,1,Ana Silva Ribeiro,Duque de Caxias,F,RJ,C
1,2,Bruna Pereira,Niterói,F,RJ,C
2,3,Túlio Nascimento,Duque de Caxias,M,RJ,S
3,4,Fernando Souza,Campinas,M,SP,S
4,5,Lúcia Andrade,São Paulo,F,SP,C
5,6,João Silva,São Paulo,None,SP,S


In [ ]:
consulta('''
select * from carro;
''','''''')

,codcarro,codmarca,modelo,valor
0,1,1,Ka,100.0
1,2,2,Argo,150.0
2,3,3,Onix,170.0
3,4,4,Polo,150.0
4,5,5,Kwid,120.0
5,6,6,Corolla,900.0
6,7,7,C-280,60000.0
7,8,7,E-320,90000.0
8,9,7,SLK-320,70000.0


In [ ]:
# Modificar valor do carro 3
ddl('''
UPDATE carro
SET valor = 200
WHERE codcarro = 3;
''')

Comando DDL executado com sucesso:

UPDATE carro 
SET valor = 200 
WHERE codcarro = 3;



In [ ]:
consulta('''
select * from carro;
''','''''')

,codcarro,codmarca,modelo,valor
0,1,1,Ka,100.0
1,2,2,Argo,150.0
2,3,3,Onix,200.0
3,4,4,Polo,150.0
4,5,5,Kwid,120.0
5,6,6,Corolla,900.0
6,7,7,C-280,60000.0
7,8,7,E-320,90000.0
8,9,7,SLK-320,70000.0


In [ ]:
# Deletar Kwid da base srsrsrsrs
ddl('''
DELETE FROM carro
WHERE modelo = 'Kwid';
''')

Comando DDL executado com sucesso:

DELETE FROM carro
WHERE modelo = 'Kwid';



In [ ]:
consulta('''
select * from carro;
''','''''')

,codcarro,codmarca,modelo,valor
0,1,1,Ka,100.0
1,2,2,Argo,150.0
2,3,3,Onix,200.0
3,4,4,Polo,150.0
4,6,6,Corolla,900.0
5,7,7,C-280,60000.0
6,8,7,E-320,90000.0
7,9,7,SLK-320,70000.0


In [ ]:
consulta('''
select * from cliente;
''','''''')

,codcliente,nome,cidade,sexo,estado,estadocivil
0,1,Ana Silva Ribeiro,Duque de Caxias,F,RJ,C
1,2,Bruna Pereira,Niterói,F,RJ,C
2,3,Túlio Nascimento,Duque de Caxias,M,RJ,S
3,4,Fernando Souza,Campinas,M,SP,S
4,5,Lúcia Andrade,São Paulo,F,SP,C
5,6,João Silva,São Paulo,None,SP,S


In [ ]:
# Deletar o cliente 5 da base aluguel
ddl('''
DELETE FROM aluguel
WHERE codcliente = '5';
''')

Comando DDL executado com sucesso:

DELETE FROM aluguel
WHERE codcliente = '5';



In [ ]:
# Deletar o cliente 5 da base cliente
ddl('''
DELETE FROM cliente
WHERE codcliente = '5';
''')

Comando DDL executado com sucesso:

DELETE FROM cliente
WHERE codcliente = '5';



In [ ]:
consulta('''
select * from aluguel;
''','''''')

,codaluguel,codcliente,codcarro,data_aluguel
0,1,3,2,2023-04-01
1,2,2,1,2023-04-02
2,3,2,1,2023-04-03
3,4,2,3,2023-04-04
4,5,1,4,2023-04-05
5,6,1,4,2023-04-13
6,7,1,1,2023-04-15
7,10,3,1,2023-04-25
8,11,1,2,2024-11-19
9,12,2,3,2024-11-18


In [ ]:
# Remover cliente 4
ddl('''
DELETE FROM cliente
WHERE codcliente = 4;
''')

Comando DDL executado com sucesso:

DELETE FROM cliente 
WHERE codcliente = 4;

